In [3]:
colors=['white','green','yellow','red','brow','pink']

In [4]:
type(colors)

list

In [5]:
colors_rdd = sc.parallelize(colors)

In [6]:
colors_rdd.collect()

['white', 'green', 'yellow', 'red', 'brow', 'pink']

# 1. Task: convert RDD to DataFrame

In [7]:
colors_rdd.map(lambda x:(x,len(x))).collect()

[('white', 5),
 ('green', 5),
 ('yellow', 6),
 ('red', 3),
 ('brow', 4),
 ('pink', 4)]

In [8]:
keyval_rdd = colors_rdd.map(lambda x:(x,len(x)))

In [9]:
# Convert from RDD to DataFrame
colors_df = keyval_rdd.toDF(['color','length'])

In [10]:
colors_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
|  brow|     4|
|  pink|     4|
+------+------+



In [11]:
print(type(colors_rdd),type(colors_df))

<class 'pyspark.rdd.RDD'> <class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
colors_df

DataFrame[color: string, length: bigint]

In [13]:
colors_df.printSchema()

root
 |-- color: string (nullable = true)
 |-- length: long (nullable = true)



In [14]:
colors_df.count()

6

In [15]:
colors_df.show(2)

+-----+------+
|color|length|
+-----+------+
|white|     5|
|green|     5|
+-----+------+
only showing top 2 rows



In [16]:
colors_df.drop('length').show()

+------+
| color|
+------+
| white|
| green|
|yellow|
|   red|
|  brow|
|  pink|
+------+



In [17]:
colors_df.show()

+------+------+
| color|length|
+------+------+
| white|     5|
| green|     5|
|yellow|     6|
|   red|     3|
|  brow|     4|
|  pink|     4|
+------+------+



In [18]:
colors_df.filter(colors_df.length < 4 ).show()

+-----+------+
|color|length|
+-----+------+
|  red|     3|
+-----+------+



In [19]:
colors_df.filter((colors_df.length > 4 ) & (colors_df[0] != 'white')).show()

+------+------+
| color|length|
+------+------+
| green|     5|
|yellow|     6|
+------+------+



In [20]:
colors_df.sort(colors_df.length.desc(),colors_df.color.asc()).show()

+------+------+
| color|length|
+------+------+
|yellow|     6|
| green|     5|
| white|     5|
|  brow|     4|
|  pink|     4|
|   red|     3|
+------+------+



In [21]:
colors_df.groupBy('length').count().show()

+------+-----+
|length|count|
+------+-----+
|     6|    1|
|     5|    2|
|     3|    1|
|     4|    2|
+------+-----+



In [22]:
colors_df.describe().show()

+-------+------+------------------+
|summary| color|            length|
+-------+------+------------------+
|  count|     6|                 6|
|   mean|  null|               4.5|
| stddev|  null|1.0488088481701516|
|    min|  brow|                 3|
|    max|yellow|                 6|
+-------+------+------------------+



# Additional Question #

In [60]:
from pyspark.sql.functions import *

In [70]:
max_value = colors_df.agg({'length':'max'}).collect()[0][0]
min_value = colors_df.agg({'length':'min'}).collect()[0][0]

In [71]:
sparkf_MinMaxNorm = udf(lambda x : (x-min_value)/(max_value-min_value))

In [72]:
colors_df.withColumn('MinMax Norm',sparkf_MinMaxNorm(col('length'))).show()

+------+------+------------------+
| color|length|       MinMax Norm|
+------+------+------------------+
| white|     5|0.6666666666666666|
| green|     5|0.6666666666666666|
|yellow|     6|               1.0|
|   red|     3|               0.0|
|  brow|     4|0.3333333333333333|
|  pink|     4|0.3333333333333333|
+------+------+------------------+

